In [1]:
import tensorflow as tf

In [2]:
import numpy as np
MODEL_LOCATION='sydney_category_model'

JSON_LOCATION=''
WEIGHTS_LOCATION=''

In [3]:
#tf.enable_eager_execution
#import tensorflow as tf
#tf.compat.v1.enable_eager_execution()
if MODEL_LOCATION!='':
    from keras.models import load_model
    model=load_model(MODEL_LOCATION)
    print("loaded model from MODEL_LOCATION")
elif JSON_LOCATION!='':
    from keras.models import model_from_json
    json_file=open(JSON_LOCATION,'r')
    loaded_model_json=json_file.read()
    json_file.close()
    model=model_from_json(loaded_model_json)
    model.load_weights(WEIGHTS_LOCATION)
    print("loaded model from JSON_LOCATION AND WEIGHTS_LOCATION")

loaded model from MODEL_LOCATION


In [23]:
#from keras import backend as K
#K.set_learning_phase(0)
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants
import tensorflow as tf
import tensorflow.python.keras.backend as K
# Disable eager execution
#tf.compat.v1.disable_eager_execution()

#with tf.compat.v1.Graph().as_default(): 

# Note: This directory structure will need to be followed 
model_version = '1'
export_dir = 'export/Servo/' + model_version

# Build the Protocol Buffer SavedModel at path defined by export_dir variable
builder = builder.SavedModelBuilder(export_dir)

with tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph()) as sess:

# Create prediction signature to be used by TensorFlow Serving Predict API
    signature = predict_signature_def(inputs={"inputs": model.input}, outputs={"score": model.output})
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

AssertionError: Export directory export/Servo/1 already exists, and isn't empty. Please choose a different export directory, or delete all the contents of the specified directory.

In [11]:
model_path = 'export_10/Servo/1/'
!saved_model_cli show --all --dir {model_path}


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 768)
        name: flatten_input
  The given SavedModel SignatureDef contains the following output(s):
    outputs['score'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 7)
        name: dense_2/Softmax:0
  Method name is: tensorflow/serving/predict
The MetaGraph with tag set ['serve'] contains the following ops: set()


In [12]:
import tarfile
model_archive = 'model.tar.gz'
with tarfile.open(model_archive, mode='w:gz') as archive:
    archive.add('export', recursive=True) 

import sagemaker
sess=sagemaker.Session()
# upload model artifacts to S3
model_data = sess.upload_data(path=model_archive, key_prefix='model')
model_data

's3://sagemaker-us-east-2-993514063544/model/model.tar.gz'

In [13]:
import sagemaker

sagemaker_session=sagemaker.Session()
#inputs = sagemaker_session.upload_data(path='model.tar.gz',key_prefix='model')

In [14]:
print("Bucket name is")
sagemaker_session.default_bucket()

Bucket name is


'sagemaker-us-east-2-993514063544'

In [15]:
!touch train.py

In [16]:
model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz'
model_data

's3://sagemaker-us-east-2-993514063544/model/model.tar.gz'

In [17]:
import boto3, re
from sagemaker import get_execution_role

role=get_execution_role()

from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                 role=role,
                                 framework_version = '2.13.0',
                                 entry_point = 'train.py')

In [19]:
%%time
health_check_timeout = 300
# from tensorflow.python.client import session

# with session.Session(graph=sagemaker_model.graph) as sess:
sagemaker_predictor = sagemaker_model.deploy(initial_instance_count=1,
                                  instance_type = 'ml.g4dn.4xlarge',
                                  endpoint_name='sydneycategorymapping-latest-test')

------------------------------------------------------*

UnexpectedStatusException: Error hosting endpoint sydneycategorymapping-latest-test: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint..

In [ ]:
##Converting the text to tensors

In [1]:
import tensorflow
from transformers import TFBertModel, BertTokenizer
#from transformers import BertTokenizer, BertModel
tokenizer= BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [7]:
import tensorflow as tf
def tokenize_bert_embeddings(texts):
    inputs=tokenizer(texts,max_length=40,truncation=True,padding="max_length",return_tensors="tf")
    outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state
    embeddings= tf.reduce_mean(embeddings,axis=1)
    return embeddings

In [8]:
text="'alcohol/substance - inpatient professional alcohol/substance abuse'"

In [9]:
embeddings = tokenize_bert_embeddings(text)

In [10]:
tensors= tf.convert_to_tensor(embeddings)

In [ ]:
##Invoking the endpoint

In [2]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
#from sagemaker.tensorflow import NumpySerializer
from sagemaker.serializers import JSONSerializer
sagemaker_session = sagemaker.Session()
endpoint_name="sydneycategorymapping-latest"

predictor = sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name,sagemaker_session)#,serializer=JSONSerializer())#,sagemaker_session)
#predictor_1 = sagemaker.predictor.Predictor(endpoint_name=endpoint_name)

In [3]:
predictor

In [4]:
import numpy as np

data = tensors.numpy()
import json

NameError: name 'tensors' is not defined

In [16]:
data=data.tolist()

In [17]:
predictor.predict([data])

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "{    "error": "Session was not created with a graph before Run()!"}". See https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logEventViewer:group=/aws/sagemaker/Endpoints/sydneycategorymapping-latest in account 993514063544 for more information.